# Requests + JSON using guide

Для работы с API чаще всего придётся пользоваться именно такой связок двух библиотек. 

Пожалуй начнём с их импорта

In [1]:
import json
import requests

Практически всегда (не помню когда не так) нам пригодится посылать <mark>.get</mark> запросы на сервер для получения ответа. Давайте разберём работу, например, запрос к сайту с картами https://en.mapy.cz/

Один из внутренних его запросов содержит внутренние параметры:
https://pro.mapy.cz/suggest/?count=5&phrase=metro&lon=37.2652&lat=55.6784&zoom=7&enableCategories=1&lang=en&personalize=1&includeNonEntityTypes=1&userLat=55.6784&userLon=37.2652&userAccuracy=20000

In [2]:
url = 'https://pro.mapy.cz/suggest/?count=5&phrase=metro&lon=37.2652&lat=55.6784&zoom=7&enableCategories=1&lang=en&personalize=1&includeNonEntityTypes=1&userLat=55.6784&userLon=37.2652&userAccuracy=20000'
page = requests.get(url)

print(page)

<Response [200]>


Взглянем-ка на то, что вернул сервер



In [7]:
content = page.text
print(type(content), content[:100] + '...', sep ='\n\n')

<class 'str'>

{"deletedFromBack":0,"hasGeo":0,"hasService":0,"id":"8f2ea956-1f15-4556-8319-b36c3fac1f14","result":...


Это очень похоже на словарь, однако сейчас это строка. Чтобы это дело исправить нужно лишь понять, что перед нами JSON-файл в естественной среде обитания, а его хорошо парсит ит превращает в классический словарь/список метод библиотеки json.

Воспользуемся методом <mark>.loads</mark> (load-string) для того чтобы работать с возвращаемым значением

In [20]:
data = json.loads(content)

print(type(data), data.keys(), sep='\n\n')

<class 'dict'>

dict_keys(['deletedFromBack', 'hasGeo', 'hasService', 'id', 'result', 'sortToUserOnCategory'])


Теперь с этими данными можно работать как с обычным словарём.

In [21]:
res = data['result'][0]

print(res)

res['userData']['bbox'] = (None,None)

print(res)

{'category': 'pubtran_osm', 'highlight': [], 'sentence': '', 'userData': {'bbox': [55.63261902819228, 37.32790265526835, 55.636422066867894, 37.339131596319845], 'correctedResult': False, 'country': '', 'district': '', 'evidenceNumber': '', 'hasAddress': False, 'highlight': [0, 5], 'highlightSecond': [], 'houseNumber': '', 'iconType': 'default', 'id': 1071044353, 'img': '', 'importance': 0.140161, 'latitude': 55.634520593672576, 'longitude': 37.33351712579409, 'mmid': '', 'mmsource': '', 'mmtype': '', 'muniId': '', 'municipality': '', 'nuts': '', 'poiType': 'poi6001', 'poiTypeId': 6001, 'popularity': 0, 'premiseIds': [], 'quarter': '', 'region': '', 'relevance': 0.7496, 'ruianId': 0, 'source': 'osm', 'street': '', 'streetNumber': '', 'suggestFirstRow': 'Metro Rasskazovka', 'suggestSecondRow': 'Bus stop, Vnukovskoye Settlement, Russia', 'suggestThirdRow': '', 'ward': '', 'wikiId': 'Q0', 'zipCode': ''}}
{'category': 'pubtran_osm', 'highlight': [], 'sentence': '', 'userData': {'bbox': (No

Ну и сохраняется всё это дело обычным методом <mark>json.dump</mark>. Например 

In [22]:
with open('backup.json', 'w') as gate:
    # json.dump(What to save?, Where to save?)
    json.dump(res, gate)

Типы данных, что могут храниться в JSON:
- Список
- Словарь
- Число
- True/False
- Строка
- null

Всё что к этому не относится (как кортеж None'ов в последнем примере), будет сконвертированно в один из возможных типов (в список null'ов), если это возможно